In [14]:
__file__ = "__init__.py"

In [ ]:
import sys, os
import warnings
import json5
import numpy as np
from pathlib import Path
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from collections import Counter
from torch.utils.data import Dataset, DataLoader



from transformers import T5Tokenizer
#     
#     T5ForConditionalGeneration,
#     Trainer,
#     TrainingArguments,
#     DataCollatorForSeq2Seq,
#     EarlyStoppingCallback,
#     TrainerCallback
# )
# import torch

warnings.filterwarnings("ignore")

project_root = Path(__file__).resolve().parents[1]
sys.path.append(str(project_root))

from utils.qgene import generate_text

In [23]:
paths = {
    "processed": os.path.abspath(f"{project_root}/data/storage/processed"),
    "qfragments": os.path.abspath(f"{project_root}/intents/qfragments.json"),
    "questions": os.path.abspath(f"{project_root}/intents/questions.csv"),
    "qtrain": os.path.abspath(f"{project_root}/intents/qtrain.csv"),
    "models": os.path.abspath(f"{project_root}/models/t5-small"),
    "results": os.path.abspath(f"{project_root}/training/results"),
}

# os.makedirs(paths["models"], exist_ok=True) 

In [ ]:
def _dataset(self, datanum: int = 1000) -> pd.DataFrame:
    if os.path.exists(paths["qtrain"]):
        old_data = pd.read_csv(paths["qtrain"])
        new_questions = pd.concat([old_data, generate_text(datanum)])
    else:
        new_questions = generate_text(datanum)
    
    new_questions.to_csv(paths["qtrain"], index=False)
    return new_questions

In [ ]:
def build_union_vocab(t5_tokenizer: T5Tokenizer, df: pd.DataFrame) -> dict:
    """
    Xây dựng từ điển union giữa từ điển của T5 tokenizer và các token lấy từ cột 'question' của df.
    Dữ liệu của generate_text được xử lý bằng cách tách theo khoảng trắng.
    """
    tokens_set = set()
    for text in df["question"].tolist():
        tokens = text.strip().split()
        tokens_set.update(tokens)
    # Lấy từ điển của T5 tokenizer
    t5_vocab = set(t5_tokenizer.get_vocab().keys())
    # Union của cả hai tập token
    union_tokens = t5_vocab.union(tokens_set)
    # Sắp xếp và tạo mapping token -> index
    vocab = {token: idx for idx, token in enumerate(sorted(union_tokens))}
    return vocab

In [26]:
class Word2VecDataset(Dataset):
    def __init__(self, corpus, window_size=5):
        """
        corpus: list of list of token indices (sử dụng union_vocab)
        window_size: kích thước cửa sổ context
        """
        self.pairs = []
        for sentence in corpus:
            sent_len = len(sentence)
            for i, center in enumerate(sentence):
                # Lấy các từ trong cửa sổ xung quanh (ngoại trừ từ trung tâm)
                for j in range(max(0, i - window_size), min(sent_len, i + window_size + 1)):
                    if i != j:
                        self.pairs.append((center, sentence[j]))
    
    def __len__(self):
        return len(self.pairs)
    
    def __getitem__(self, idx):
        return self.pairs[idx]

In [38]:
corpus = [
    [2, 3, 0],          # câu: "this is hello"
    [0, 1, 2, 3, 4],    # câu: "hello world this is test"      
]

# Sử dụng window_size=2 để lấy các cặp
dataset = Word2VecDataset(corpus, window_size=2)
print("Số lượng cặp:", len(dataset))
for i in range(min(5, len(dataset))):
    print(f"Cặp {i}: {dataset[i]}")

Số lượng cặp: 20
Cặp 0: (2, 3)
Cặp 1: (2, 0)
Cặp 2: (3, 2)
Cặp 3: (3, 0)
Cặp 4: (0, 2)


In [39]:
class Word2VecModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(Word2VecModel, self).__init__()
        # embedding cho từ trung tâm và context
        self.in_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.out_embeddings = nn.Embedding(vocab_size, embedding_dim)
    
    def forward(self, center, context, negative_context):
        # center: (batch,)
        # context: (batch,)
        # negative_context: (batch, neg_samples)
        center_embed = self.in_embeddings(center)           # (batch, embedding_dim)
        context_embed = self.out_embeddings(context)         # (batch, embedding_dim)
        # Tính điểm cho cặp (center, context) dương
        score = (center_embed * context_embed).sum(dim=1)
        log_target = torch.log(torch.sigmoid(score) + 1e-10)
        
        # Tính điểm cho các cặp âm (negative samples)
        neg_embed = self.out_embeddings(negative_context)    # (batch, neg_samples, embedding_dim)
        neg_score = torch.bmm(neg_embed, center_embed.unsqueeze(2)).squeeze()
        log_negative = torch.log(torch.sigmoid(-neg_score) + 1e-10).sum(dim=1)
        
        loss = -(log_target + log_negative).mean()
        return loss
    
    def get_input_embeddings(self):
        return self.in_embeddings.weight.data.cpu().numpy()

In [ ]:
b